In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [4]:
!chmod 600 /content/kaggle.json

In [90]:
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

  0% 0.00/1.28M [00:00<?, ?B/s]
100% 1.28M/1.28M [00:00<00:00, 42.4MB/s]
sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/494k [00:00<?, ?B/s]
100% 494k/494k [00:00<00:00, 68.8MB/s]


In [91]:
!unzip \*.zip && rm *.zip

Archive:  test.tsv.zip
replace test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.tsv                

Archive:  train.tsv.zip
replace train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.tsv               

2 archives were successfully processed.


In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
train = pd.read_csv('/content/train.tsv',sep='\t')
test = pd.read_csv('/content/test.tsv',sep='\t')

In [94]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [95]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [96]:
train.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [97]:
#5 sentiments possible

In [98]:
train.SentenceId.value_counts()

1       63
5555    63
509     59
625     58
403     57
        ..
8451     1
5718     1
3560     1
1666     1
1178     1
Name: SentenceId, Length: 8529, dtype: int64

In [99]:
train.SentenceId.nunique()

8529

In [100]:
train.shape

(156060, 4)

In [101]:
test.shape

(66292, 3)

In [102]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [103]:
train['text_len'] = train['Phrase'].apply(len)
test['text_len'] = test['Phrase'].apply(len)

In [104]:
from nltk.stem.porter import PorterStemmer
import re

In [105]:
from nltk.corpus import stopwords

In [106]:
ps = PorterStemmer() #pour "text preprocessing"

message = []

for i in range(0, train.shape[0]):
    #accepter que les mots alphabétiques
    review = re.sub('[^a-zA-Z]', ' ', train['Phrase'][i])
    #convertir tous minuscule
    review = review.lower()
    #splitter chaque ligne
    review = review.split()
    #
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #construire de nouveau la ligne
    review = ' '.join(review)
    message.append(review)

In [108]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)
X = cv.fit_transform(message).toarray()

In [109]:
Y=np.array(train['Sentiment'])

In [110]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [111]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=42)
rfc.fit(x_train,y_train)
y_pred = rfc.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6359092656670511

In [112]:
ps1 = PorterStemmer() #pour "text preprocessing"

message1 = []

for i in range(0, test.shape[0]):
    #accepter que les mots alphabétiques
    review1 = re.sub('[^a-zA-Z]', ' ', test['Phrase'][i])
    #convertir tous minuscule
    review1 = review1.lower()
    #splitter chaque ligne
    review1 = review1.split()
    #
    review1 = [ps1.stem(word) for word in review1 if not word in stopwords.words('english')]
    #construire de nouveau la ligne
    review1 = ' '.join(review1)
    message1.append(review1)

In [113]:
cv1 = CountVectorizer(max_features=3000)
X1 = cv.fit_transform(message1).toarray()

In [114]:
Y1 = rfc.predict(X1)

In [115]:
sub = pd.read_csv('/content/sampleSubmission.csv')
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [117]:
submission = pd.DataFrame()
submission['PhraseId'] = test.PhraseId
submission['Sentiment'] = Y1
submission.to_csv('Random_Forest_V1.csv', index=False)